In [2]:
############################
##- Installing libraries -##
#--------- Python 3 -------#
############################
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain

def install(package):
    pipmain(['install', "--upgrade", "pip"])
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        try:
            pip._internal.main(['install', package])
        except:
            print(pip._internal.main(['install', package]))

###########################
##- Installing Biopython-##
###########################
install('biopython')

Requirement already up-to-date: pip in /home/anajulia/anaconda2/lib/python2.7/site-packages (18.1)


In [ ]:
###################################################
##-- checking if clustalw is already installed --##
#  Source: https://askubuntu.com/questions/35744  #
###################################################

##-- def query_package --##
import os.path
path = '/usr/bin/clustalw'
def query_package(path):
    if os.path.exists(path):
        return 'clustalw is ready to use'
    else:
        return 'please install clustalw'

##-- def _install_clustalw --##
def _install_clustalw():
    subprocess.call("sudo get install clustalw", shell=True)

##-- check point --##
if query_package(path) == "clustalw is ready to use":
    print('clustalw is ready to use')
    pass
else:
    print('Installing clustalw')
    _intall_clustal()
    print('Done')

In [1]:
### IMPORTANT DATA INFO:this script takes '.txt' files with fasta sequences of an ensemble of proteins
## IMPORTANT DATA INFO: The reference sequence must be the first file sequence 
#############################################################################################################

##- Importing linbraries -##
############################
import os
import glob
import subprocess

#- Setting the working directory -##
####################################
path = '/home/anajulia/Be_project/fastasequences'
os.chdir(path)

In [12]:
#- Spliting sequences into separates files -##
##############################################
files_list = glob.glob('*.txt')
for i in files_list:
    subprocess.call(['split','-l2','-d',i,i.split('_')[0]+'_sequence.'])

In [72]:
##- Importing libraries -##
###########################
from Bio import pairwise2
from Bio import SeqIO
from difflib import SequenceMatcher
try:
    from StringIO import StringIO # Python 2
except ImportError:
    from io import StringIO # Python 3
import itertools

#- reading files -##
####################
all_files = os.listdir(path)
files_list = glob.glob('*.txt')

#- Sequence match dict -##
match_dict = {}
#- Sequence matcher all to all -##
##################################
for i in files_list:
    ped_code = i.split('_')[0]
    seqs_list = []
    for j in all_files:
        if j.startswith(ped_code):
            if j.endswith('.txt') == False:
                seqs_list.append(j)

    ##- All the combinations possible -##
    combinations = list(itertools.combinations(seqs_list, 2))
    if len(combinations) > 1:
        for x in list(set(combinations)):
                #######-- Aligning sequences --########
                data1 = SeqIO.read(x[0], "fasta")
                data2 = SeqIO.read(x[1], "fasta")
                ##- ratio of coincidence between sequences -##
                s = SequenceMatcher(None, data1.seq,data2.seq)
                ##- Saving the info in a dict -##
                match_dict[x] = s.ratio()

##--- Building a score similarity table --##
############################################

#-- importing pandas --#
import pandas as pd

#-- Creating a table from the dict --#
df = pd.DataFrame(match_dict.items(), columns=['Pair', 'match_score'])
#-- Rewriting the Pair column --#
pares = [str(x).replace('(','').replace(')','').replace(',','-').replace("'",'') for x in df['Pair'].tolist()]
df['Pair'] = pares

In [67]:
#- Checking the score in order to know which PEDs we have to be plotted -#
#- getting the PEDs ids -#
to_be_plotted = []
ped_acc = []
for i in df['Pair'].tolist():
    ped_acc.append(i.split('-')[0].split('_')[0])
ped_acc = list(set(ped_acc))

#- checking whithin ensembles -#
pairs_to_plot =  []
for i in ped_acc:
    pairs = []
    for j in df['Pair'].tolist():
        if j.startswith(i):
            ids = j.split('-')
            pairs.append(j)
    pairs = list(set(pairs))
    for x in range(0,len(pairs)):
        if '01' in pairs[x]:
            for j in pairs[x].replace(' ','').split('-'):
                if '00' in j:
                    pairs_to_plot.append(pairs[x])
                if '01' not in j:
                    if '00' not in j:
                        if df[df['Pair'] == pairs[x]]['match_score'].iloc[0] < 1:
                            pairs_to_plot.append(pairs[x])

In [86]:
######################################
##-- Creating the a new directory --##
######################################
dirName = 'pairwise_alig_plot'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " + dirName + " Created ") 
except FileExistsError:
    print("Directory " + dirName +  " already exists")

#- Aligment plots -#
####################
for i in pairs_to_plot:
    seq1 = i.replace(' ','').split('-')[0]
    seq2 = i.replace(' ','').split('-')[1]
    #######-- Aligning sequences --########
    data1 = SeqIO.read(seq1, "fasta")
    data2 = SeqIO.read(seq2, "fasta")
    ##-- Align --##
    alignments = pairwise2.align.globalxx(data1.seq,data2.seq)
    ##-- Ploting aligments --##
    aln = pairwise2.format_alignment(*alignments[0])
    #-- Building an output --#
    path_out = path +'/pairwise_alig_plot/'+ seq1.split('_')[0] + seq1.split('_')[-1][-2:] +'-'+ seq2.split('_')[0] + seq2.split('_')[-1][-2:] +'_aln_pair.plot'
    path_out_fasta = path +'/'+ seq1.split('_')[0]+ seq1 .split('_')[-1][-2:] +'-'+ seq2.split('_')[0] + seq2.split('_')[-1][-2:] +'_pair.fasta'
    ##-- Writing outputs --##
    with open(path_out,'a') as miarch:
        miarch.write(aln)

#- Pairwise aligments all to all -##
####################################
##- Version with all the combinations possible -##
##################################################
for i in files_list:
    ped_code = i.split('_')[0]
    seqs_list = []
    for j in all_files:
        if j.startswith(ped_code):
            if j.endswith('.txt') == False:
                seqs_list.append(j)
                
    combinations = list(itertools.combinations(seqs_list, 2))                
    if len(combinations) > 1:
        for x in combinations:
            ##-- we take only the combinations including the uniprot seq ref
            if '00' in str(x):
                #######-- Aligning sequences --########
                data1 = SeqIO.read(x[0], "fasta")
                data2 = SeqIO.read(x[1], "fasta")
                ##-- Align --##
                alignments = pairwise2.align.globalxx(data1.seq,data2.seq)
                ##-- Ploting aligments --##
                aln = pairwise2.format_alignment(*alignments[0])
                #-- Building an output --#
                path_out = path +'/'+ x[0].split('_')[0]+x[0].split('_')[-1][-2:] +'-'+ x[1].split('_')[0] + x[1].split('_')[-1][-2:] +'_aln_pair.plot'
                path_out_fasta = path +'/'+ x[0].split('_')[0]+x[0].split('_')[-1][-2:] +'-'+ x[1].split('_')[0] + x[1].split('_')[-1][-2:] +'_pair.fasta'
                #print(path_out_fasta)
                #print(aln)
                ##-- Writing outputs --##
                #with open(path_out,'a') as miarch:
                    #miarch.write(aln)
                    
#########################################
##-- Moving files to a new directory --##
#########################################
dirName = 'pairwise_alig_plot'
try:
    # Create target Directory
    os.mkdir(dirName)
    print("Directory " + dirName + " Created ") 
except FileExistsError:
    print("Directory " + dirName +  " already exists")

###########################
##-- Moving plot files --##    
source = path +'/'
dest1 = path + '/' + dirName

##-- Listing files --##
files = glob.glob('*.plot')
##-- Moving files --##
import shutil
for f in files:
        shutil.move(source + f, dest1)
print('All the plot files were moved correctly')